In [1]:
# third-party IPython extensions:
%load_ext line_profiler
%load_ext memory_profiler

import functools
import inspect
import os
from timeit import timeit

# local import:
import compare_funcs as cf

# Optimization

Most of the basic operations that our Python programs carry out are executed so quickly as to seem instantaneous. But because of the way that computers work, and because of the way that Python itself is designed, some basic operations take longer to execute in Python than do others. All else being equal, if we have the choice between two or more methods of accomplishing the same task, we would prefer the method that runs the fastest. Here we will look at a few techniques that can help us make our programs faster.

Before doing so, it is important to note that all else is rarely equal. For many purposes it is much more important for our programs to be flexible, quick to write and to modify, and above all clear for others to read, than for them to be fast. We should avoid wasting time trying to make our program run a few milliseconds faster if this difference is not crucial.

Cases in which we might genuinely need faster code include:

* we are running the program on large inputs, such as huge squigabyte-sized data files
* our program runs on a server and is queried by squillions of users per second

In such cases, we may need to test different candidate solutions to some tasks, to see whether any are usefully faster for the specific cases that our program has to deal with. We may then make some changes, so as to optimize our program for these cases.

The `timeit` module from the Python standard library provides a function that times the execution of a statement. We can enter the statement as a string. The units of the return value are seconds.

Here for example to time a comparison of two integers:

In [2]:
timeit('9000 > 0')

0.030683400997077115

This looks suspiciously slow for such a simple operation, almost as slow as a human being answering the same question. But this is in fact the execution time for a million repetitions of the statement. `timeit()` takes a `number` argument specifying how many times to run the statement, and the return value is the total execution time for all repetitions. The default is 1000000, as we can see from the function help:

In [3]:
help(timeit)

Help on function timeit in module timeit:

timeit(stmt='pass', setup='pass', timer=<built-in function perf_counter>, number=1000000, globals=None)
    Convenience function to create Timer object and call timeit method.



This is a useful feature, since the execution time of a single repetition of a simple command is often too small to be easily interpretable.

In [4]:
timeit('9000 > 0', number=1)

5.589972715824842e-07

Python commands inside strings are not always easy to read, and are somewhat error-prone. `timeit()` allows for the first argument to instead be a function. This allows us to time a longer sequence of commands in a clearer way.

Here with a somewhat spurious example function, just for illustration:

In [5]:
def get_even():
    numbers = []
    for i in range(100):
        numbers.append(i)
    evens = []
    for n in numbers:
        if n % 2 == 0:
            evens.append(n)
    return evens

timeit(get_even)

12.001796001000912

This function will be quite slow if we need it millions of times. It contains quite a few inefficiencies, such as assigning and appending to a list in a loop when the list entries could be created using built-in functions, and iterating over the same set of numbers twice. Below is an alternative that converts the desired range of numbers directly into a list. The gain in speed over a million calls is big.

(We should also make sure to check first that the return value of the alternative is really the same.)

In [6]:
def get_even_alt():
    return list(range(0, 100, 2))

get_even_alt() == get_even()

True

In [7]:
timeit(get_even_alt)

0.6487632889984525

The `compare_funcs` module provided along with this notebook defines a few simple functions, pairs of which accomplish the same common task in two different ways. For example, there is a function for creating a list by appending to it in a loop, and a partner function for the same task but using a list comprehension.

In [8]:
for func in [cf.make_list_with_loop, cf.make_list_with_comprehension]:
    print(inspect.getsource(func))

def make_list_with_loop(n):
    """
    Return a list of the first n square numbers.
    This version appends to the list in a loop.
    Compare with: make_list_with_comprehension
    """
    squares = []
    for i in range(n):
        squares.append(i**2)
    return squares

def make_list_with_comprehension(n):
    """
    Return a list of the first n square numbers.
    This version creates the list with a comprehension.
    Compare with: make_list_with_loop
    """
    return [i**2 for i in range(n)]



Because these functions take a required argument that allows some parameter concerning the size of the task to vary, we need to wrap them together with a value for this argument, using the `partial()` function from `functools`, in order to be able to pass them in as a variable name as the `timeit()` function requires.

In [9]:
timeit(functools.partial(cf.make_list_with_loop, 9000), number=1)

0.0037860869997530244

## File I/O

One of the most time-consuming everyday operations in computing is reading and writing to and from disk. In almost all cases it takes considerably more time to alter data on a permanent storage device like a computer hard drive than to alter it in temporary memory. So when writing lots of data we often need to make a compromise between ensuring our data is saved to disk often (safer, but slower), and building up data in our program's workspace and then writing it once when finished (more risk of data loss in the event of a crash, but faster).

The example module provides two alternative functions to illustrate this phenomenon.

In [10]:
for func in [cf.write_string_on_disk, cf.write_string_in_memory]:
    print(inspect.getsource(func))
    print('execution time: {} s\n\n'.format(timeit(functools.partial(func, 9000), number=1)))

def write_string_on_disk(n):
    """
    Write a string of n digits to a temporary file.
    This version writes each digit to the file in a separate open and write.
    Compare with: write_string_in_memory
    """
    with open(TEMPFILENAME, mode='w') as f:
        for i in range(n):
            f.write(str(i % 10))
            f.flush() # (ensures writing occurs on every iteration)
    os.remove(TEMPFILENAME)

execution time: 0.03712692300177878 s


def write_string_in_memory(n):
    """
    Write a string of n digits to a temporary file.
    This version first grows the string in memory then writes it to file.
    Compare with: write_string_on_disk
    """
    text = ''
    for i in range(n):
        text = text + str(i % 10)
    with open(TEMPFILENAME, mode='w') as f:
        f.write(text)
    os.remove(TEMPFILENAME)

execution time: 0.0023688319997745566 s




## Inline timing in IPython

IPython provides a [magic function](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit) `%timeit` that times the execution of a statement. This is often more convenient if we are working in IPython, as it provides a few benefits:

* the input is a statement, not a function name, so we can time functions with input arguments, nested functions, etc.
* the number of repetitions of the statement is determined automatically so as to produce a tolerable overall execution time
* the result is formatted with automatically chosen units (milliseconds, microseconds, etc.) so as to shorten extremely small decimals

In [11]:
%timeit cf.make_list_with_loop(9000)

100 loops, best of 3: 2.32 ms per loop


## Line profiling

If we are timing a more elaborate function or a program that calls many functions, it may be helpful to first get an overview of which parts take the longest to execute. Often it is just one or two lines that account for most of the execution time.

The [line_profiler](https://github.com/rkern/line_profiler) package provides functions for timing code line-by-line. It also provides an extension to IPython. This extension makes available the magic command `%lprun`, which runs a function or module and prints a line-by-line analysis. The `-f` option specifies which functions to profile (and we must specify this even if the input is just a call to that function).

In [12]:
%lprun -f cf.write_string_on_disk cf.write_string_on_disk(9000)

Somewhat frustratingly, `line_profiler`'s IPython extensions are configured so that the output always opens in the separate 'pager' frame, rather than being printed. So we cannot embed the output in a notebook without first reconfiguring the pager options (*extremely* tedious) or dumping the output into a file and then reading it in (easier, if slightly messy).

For the sake of completeness, I show here how to dump the output to file and read it back in.

In [13]:
%lprun -f cf.write_string_on_disk -T temp_profile_output.txt cf.write_string_on_disk(9000)

print(open('temp_profile_output.txt').read())


*** Profile printout saved to text file 'temp_profile_output.txt'. 
Timer unit: 1e-06 s

Total time: 0.047259 s
File: /home/lt/Dropbox/Teach/Mind_and_Brain/Courses/Python/GitHub/python-examples/profiling/compare_funcs.py
Function: write_string_on_disk at line 51

Line #      Hits         Time  Per Hit   % Time  Line Contents
    51                                           def write_string_on_disk(n):
    52                                               """
    53                                               Write a string of n digits to a temporary file.
    54                                               This version writes each digit to the file in a separate open and write.
    55                                               Compare with: write_string_in_memory
    56                                               """
    57         1        239.0    239.0      0.5      with open(TEMPFILENAME, mode='w') as f:
    58      9001       4995.0      0.6     10.6          for i in rang

## Memory usage

For a lot of applications, time is the critical factor in optimization. But there are other aspects of our program's performance that might limit its capabilities. The most common of these is the amount of temporary memory it requires. For example, we might run out of memory if:

* we have to process huge squigabyte-sized files
* we have to copy the contents of very large arrays or lists

If tasks like these take us close to the maximum amount of memory that Python is permitted to work with, then our program may sometimes fail with a `MemoryError` or not transfer well across different systems with different amounts of memory available.

The [memory_profiler](https://github.com/rkern/line_profiler) package (which was inspired by `line_profiler`) provides functions for assessing the memory usage of a function or program as it executes. It provides magic functions analogous to `%timeit` and `%lprun`.

The example module provides two more alternative functions with which we can illustrate the memory cost of copying large arrays. Each performs the same calculations on the values in an array, but one of the functions avoids allocating any intermediate arrays in the process (perhaps at some cost to the readability of the code).

In [14]:
for func in [cf.calculate_via_copies, cf.calculate_without_copies]:
    print(inspect.getsource(func))

def calculate_via_copies(n):
    """
    Return the density of the standard normal distribution for n values \
    linearly spaced between -3 and 3.
    This version allocates intermediate arrays.
    Compare with: calculate_without_copies
    """
    x = numpy.linspace(-3, 3, n)
    xSq = x**2
    halfxSq = xSq / 2
    negHalfxSq = -halfxSq
    eToTheNegHalfxSq = numpy.e**negHalfxSq
    factor = 1 / numpy.sqrt(2 * numpy.pi)
    return factor * eToTheNegHalfxSq

def calculate_without_copies(n):
    """
    Return the density of the standard normal distribution for n values \
    linearly spaced between -3 and 3.
    This version allocates no intermediate arrays.
    Compare with: calculate_via_copies
    """
    return (1 / numpy.sqrt(2*numpy.pi)) * numpy.e**-(numpy.linspace(-3, 3, n)**2 / 2)



In [15]:
%memit cf.calculate_via_copies(1000000)

peak memory: 90.56 MiB, increment: 37.69 MiB


In [16]:
%memit cf.calculate_without_copies(1000000)

peak memory: 68.99 MiB, increment: 15.68 MiB


Line-by-line memory profiling shows that memory usage increases slightly with each intermediate array.

In [17]:
%mprun -f cf.calculate_via_copies -T temp_profile_output.txt cf.calculate_via_copies(1000000)

print(open('temp_profile_output.txt').read())



*** Profile printout saved to text file temp_profile_output.txt. 
Filename: /home/lt/Dropbox/Teach/Mind_and_Brain/Courses/Python/GitHub/python-examples/profiling/compare_funcs.py

Line #    Mem usage    Increment   Line Contents
    82     54.0 MiB     54.0 MiB   def calculate_via_copies(n):
    83                                 """
    84                                 Return the density of the standard normal distribution for n values \
    85                                 linearly spaced between -3 and 3.
    86                                 This version allocates intermediate arrays.
    87                                 Compare with: calculate_without_copies
    88                                 """
    89     69.0 MiB     15.0 MiB       x = numpy.linspace(-3, 3, n)
    90     69.0 MiB      0.0 MiB       xSq = x**2
    91     76.8 MiB      7.7 MiB       halfxSq = xSq / 2
    92     84.2 MiB      7.5 MiB       negHalfxSq = -halfxSq
    93     92.0 MiB      7.7 MiB       e

In [18]:
os.remove('temp_profile_output.txt')